# Bigram Language Model

## N-grams

An n-gram is a contiguous sequence of _n_ tokens (characters, words, or subwords) from a given text.  
A 2-gram, or bigram, groups tokens into pairs.

For example, using words as tokens, in the sentence:

- `Charlie Parker is as good as Bach`

The 2-grams would be:

- `Charlie Parker`
- `Parker is`
- `is as`
- `as good`
- `good as`
- `as Bach`

> A `BigramLanguageModel` is a statistical model that estimates the probability of a token in a sequence based only on the previous token.

### Bigram Probability Function

where:

- $ P(w*n \mid w*{n-1}) $ is the **conditional probability** of the word $ w*n $ occurring given that $ w*{n-1} $ came before it.
- $ \text{Count}(w*{n-1}, w_n) $ is the number of times the word pair (bigram) $ (w*{n-1}, w_n) $ appears in the training corpus.
- $ \text{Count}(w*{n-1}) $ is the number of times the word $ w*{n-1} $ appears as a starting word in any bigram.

$$ P(w*n | w*{n-1}) = \frac{\text{Count}(w*{n-1}, w_n)}{\text{Count}(w*{n-1})} $$

**How does the model know which word comes next?**

The model is trained on structured data (our bigrams). It learns that some token pairs are more likely to occur together than others. When it sees a token, it calculates the probability of the next token based on how frequently they appeared together in training data.

## Disclaimer

Since we want to build an LLM from scratch, our approach is not based on a traditional probability function. Instead, our model uses a neural network with an embedding layer whose weights are updated during each training step, and this is what the following code implements.


In [ ]:
# print the env being used
import sys
print(sys.executable)

import torch
import torch.nn as nn
from torch.nn import functional as F

# Just to check this damn library works XD
import pylzma
# print('pylzma is working')

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# print(f'Device :{device}')

## Parameters
block_size = 8
batch_size = 4

### Character-based BigramLanguageModel

In this example we are going to build a characted-based BLM. Our tokens will be single chars. It is not a great model, but simplifies the example.
So, open a file, define a set of tokens (the unique characters appearing in that file) and stablish a vocab_size, which is just the size of this set.


In [ ]:
# Open some data
with open ('./data/ecce_homo.txt', 'r', encoding='utf-8') as f: 
    text = f.read()

char_set = sorted(set(text))
vocab_size = len(char_set)

### Encoding and Decoding

Language models operate on numbers, so we need a way to transform characters to numbers and viceversa.
Encode recieves a string and returns an array with encoded characters.
Decode recieves an array and decodes back each number to the corresponding character in our set.


In [ ]:
# Naive text-to-int char encoding

string_to_int = { ch: i  for i, ch in enumerate(char_set)}
int_to_string = { i : ch for i, ch in enumerate(char_set)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.int64)

### Preparing Data

The data is split into:

- **Train set** → Used for training the model.
- **Validation set** → Used to test generalization (data not seen by the model before).

#### `get_batch`

This function prepares a batch (a unit of training/evaluation data) to fit into the model.

- `ix` is a randomly generated sequence of indices referencing starting points in the dataset.
- `block_size` defines the window (context size) given to the model to predict the next token.
- `x` is a tensor of shape `(batch_size, block_size)`, where:
  - Each row is a sequence of `block_size` tokens from `data`, starting at a randomly chosen index.
- `y` is a **shifted** version of `x`, meaning:
  - Each `y[j]` contains the **next tokens** of `x[j]`.
  - This allows the model to learn **the probability of the next token** at each position in `x`.

> In the current model, as we'll se, the window is not used for anything.


In [ ]:
# Prepare data

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix]) # stack converts a list of tensors into a tensor itself
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# x, y = get_batch('train')
# print('inputs: ')
# print(x)
# print('targets: ')
# print(y)

### The model

Thanks to PyTorch, we are not building entirely (or even nearly) from scratch. A `nn.Module` is a _"Base class for all neural network modules"_, and offers a set of functionality related to defining layers, managing parameters, moving models between CPU/GPU, etc.

Our model initializes with just the vocab_size and creates a token embeding table. This inocent table presented in the course as 'nah, just a lookup table' is a trainable layer of the model. It contains as many rows as `vocab_size`, each row containing `vocab_size` elements and at this point is the layer we are just training: is 'the model' itself.

This nn.Embedding layer can accept an input that is a single index, a list of indices, or even a batch of indices, and it will return the corresponding embedding for each index. An embedding will contain a list of `vocab_size` elements representing raw values which later are converted to the probabilities for each token to follow the one provided by our index.

#### Input

The input is a Tensor of shape `(B, T)`, being `B` the batch size and `T` the sequence length.
In the current's model state, the sequence is not really doing anything, since the model is not context aware, so in the `(B, T, C)`, the `T` could be 1 and nothing will change. But, maybe later in the course they introduce some cool features which can use this context.

#### Forward function

Responsible for calculating a single prediction and (if targets are provided) calculate the loss value throghout the `cross_entropy` function.

**Reshaping**
Logits have shape `(B, T, C)`, being `C` the vocab size. Due to **_Torch-Magic™_** accepting whatever you throw at it, the call to `self.token_embeding_table` accepts any `(B, T)` Tensor and returns another Tensor with the added embeddings dimension.

> `indices = [[0], [0]]` &rarr; `B = 2, T = 1`  
> `logits` &rarr; `B = 2, T = 1, C = vocab_size`

> This will return a Tensor containing 2 (`B`) sets of 1 (`T`) embeding of dimension `C`

The scary dimension-juggling is just about flattening the tensors to adapt them to expected parameters in the cross_entropy function.

> `targets = targets.view(B*T)` &rarr; if targets is `[[0, 1], [2, 3]]` is converted to `[0, 1, 2, 3]`  
> `logits = logits.view(B*T, C)` &rarr; just flat the embedings, so we get a i \* j Tensor.

#### Generate function

This function generates new tokens one at a time, up to `max_new_tokens`, by repeatedly:

1. Computing probabilities for the next token

1. Sampling a token from those probabilities

1. Adding the sampled token to the sequence

**Reshape juggling again**  
Here `logits = logits[:, -1, :]`, as we are not passing targets to the forward function we recieve a `(B, T, C)` Tensor. Then, for each batch, we take the embedding (the prediction) generated by the last token in the batch sequence.
Then softmax transforms the values in the embedding to probabilities and we take one base on a multinomial distribution.
Finally we take the generated token and append it (cat &rarr; concat) to the input, so the next generated token in every batch will be based on that last generated token.

### Why the hell the model accepts any B\*T input?

Because the model can be trained in parallel. Again, thanks to Torch-Magic™, operations like softmax and multinomial sampling automatically adapt to the input dimensions. This means you can throw various shapes at these functions, and they'll "just work," handling the dimension broadcasting and manipulation behind the scenes.


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embeding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # predictions
            logits, _ = self.forward(index)
            # last time step
            logits = logits[:, -1, :] # (B, T, C) =>  (B, C)
            # softMax
            probs = F.softmax(logits, dim=-1) # (B, C), calculates probabilities on the C axis, the embeddings
            # sample from distribution
            index_next  = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim = 1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context)
# generated_chars = decode(m.generate(context, max_new_tokens=1)[0].tolist())
# print(generated_chars)


In [ ]:
eval_iters = 500

# Just a function to evaluate the model on the fly, while it is training
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


## Training

We have a model but is not trained. For this we need to optimize the lost function.
Torch exposes a bunch of optimizers better suited for one or another task.

To train, just iterate over:

- Get some train data (input, expected).
- Call model.forward to get the predictions and loss.
- Reset gradients to not accumulate them between iterations.
- Apply backward propagation.
- Update parameters (step).


In [ ]:
# PyTorch optimizer
learning_rate = 3e-1
dropout = 0.2
max_iters = 10000
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for item in range(max_iters):
    if item % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {item}\t | train_loss: {losses['train']:.4f} | val_loss: {losses['val']:.4f}")
    # sample batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

# Predict

And this is pretty much all, we trained a model based on a newral network with one embedding layer to generate some predictions.


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=200)[0].tolist())
print(generated_chars)